In [2]:
import datetime
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt

# Index¶
1.Load the data
2.Hypothesis test on all data
3.Hypothesis test grouped by country
4.daily spent change in each country
5.daily customers change in each country
6.daily transactions change in each country
7.Country-based conclusion
8.Answer question 1
9.Answer question 2
10.Answer question 3

In [5]:
refer = pd.read_csv('referral.csv')
refer.date = pd.to_datetime(refer.date)
del refer['device_id']

In [6]:
refer.head(5)

,user_id,date,country,money_spent,is_referral
0,2,2015-10-03,FR,65,0
1,3,2015-10-03,CA,54,0
2,6,2015-10-03,FR,35,0
3,7,2015-10-03,UK,73,0
4,7,2015-10-03,MX,35,0


In [7]:
start_time = datetime.datetime(2015,10,31)

In [9]:
refer.date.describe()

count                   97341
unique                     56
top       2015-11-14 00:00:00
freq                     3303
first     2015-10-03 00:00:00
last      2015-11-27 00:00:00
Name: date, dtype: object

In [11]:
(pd.Series(refer.date.unique()) >= start_time).value_counts()

True     28
False    28
dtype: int64

In [38]:
def count_spent(df):
    d = {}
    d['n_purchase'] = df.shape[0]# number of purchase in that day
    d['total_spent'] = df.money_spent.sum()# total money spent in that day
    d['n_customer'] = df.user_id.unique().shape[0]# how many customers access the store that day
    return pd.Series(d)

In [41]:
def daily_stats(df):
    """
    given a dataframe
    1.  group by day, and return '#purchase','total spent money','#customers' on each day
    2.  split daily data into two groups, before the program and after the program
    3.  for each 'sale index' ('#purchase','total spent money','#customers'), 
        calculate the mean before/after the program, their difference, and pvalue 
    """
    grpby_day = df.groupby('date').apply(count_spent)
    
    grpby_day_before = grpby_day.loc[grpby_day.index < start_time,:]
    grpby_day_after = grpby_day.loc[grpby_day.index >= start_time,:]
    
    
    d = []
    cols = ['n_purchase','total_spent','n_customer']
    for col in cols:
        pre_data = grpby_day_before.loc[:,col]
        pre_data_mean = pre_data.mean()
        
        post_data = grpby_day_after.loc[:,col]
        post_data_mean = post_data.mean()
        
        result = ss.ttest_ind(pre_data,post_data,equal_var = False)
        pvalue = result.pvalue/2
        
        d.append({'pre_mean':pre_data_mean,'post_mean':post_data_mean,'mean_diff':post_data_mean-pre_data_mean,'pvalue':pvalue})
    return pd.DataFrame(d,index = cols).loc[:,['pre_mean','post_mean','mean_diff','pvalue']]
        
    
    
    

In [42]:
daily_stats(refer)

,pre_mean,post_mean,mean_diff,pvalue
n_purchase,1690.750000,1785.714286,94.964286,0.348257
total_spent,71657.000000,83714.392857,12057.392857,0.135194
n_customer,1384.464286,1686.964286,302.500000,0.059545


although after launching the 'user referral' program, in all three 'sale index', i.e., 'daily purchase activity', 'daily money spent', 'daily customers', are all increased, however, none of those increment are significant. (by using a 0.05 significant level)



Check Country Influcence

In [44]:
refer.country.describe()

count     97341
unique        9
top          UK
freq      15493
Name: country, dtype: object

In [47]:
refer.country.value_counts()

UK    15493
FR    15396
US    15280
IT    11446
DE    11093
ES     9831
CA     9440
MX     8133
CH     1229
Name: country, dtype: int64

In [48]:
daily_stats_by_country = refer.groupby('country').apply(daily_stats)
daily_stats_by_country

pre_mean     post_mean    mean_diff    pvalue
country                                                               
CA      n_purchase     177.142857    160.000000   -17.142857  0.233985
        total_spent   7468.428571   7880.428571   412.000000  0.351704
        n_customer     173.285714    159.178571   -14.107143  0.268256
CH      n_purchase      26.821429     17.071429    -9.750000  0.003072
        total_spent   1536.321429   1023.892857  -512.428571  0.006941
        n_customer      26.714286     17.071429    -9.642857  0.003142
DE      n_purchase     232.142857    164.035714   -68.107143  0.011798
        total_spent   9856.750000   8013.964286 -1842.785714  0.081459
        n_customer     224.964286    163.250000   -61.714286  0.015665
ES      n_purchase     156.607143    194.500000    37.892857  0.072638
        total_spent   6648.642857   8660.571429  2011.928571  0.037522
        n_customer     153.392857    193.214286    39.821429  0.057954
FR      n_purchase     244.142857    305.714286    61.571429  0.058996
        total_spent  10385.250000  13635.000000  3249.750000  0.031843
        n_customer     236.500000    302.535714    66.035714  0.041124
IT      n_purchase     180.857143    227.928571    47.071429  0.057454
        total_spent   7651.571429  10193.428571  2541.857143  0.025730
        n_customer     176.535714    226.107143    49.571429  0.043911
MX      n_purchase     126.464286    164.000000    37.535714  0.032430
        total_spent   4975.464286   7033.214286  2057.750000  0.009670
        n_customer     124.392857    163.107143    38.714286  0.026203
UK      n_purchase     264.285714    289.035714    24.750000  0.261183
        total_spent  11213.535714  14196.428571  2982.892857  0.048490
        n_customer     255.571429    286.321429    30.750000  0.204398
US      n_purchase     282.285714    263.428571   -18.857143  0.307801
        total_spent  11921.035714  13077.464286  1156.428571  0.248874
        n_customer     273.178571    261.107143   -12.071429  0.368860

In [49]:
daily_stats_by_country.xs('total_spent',level = 1).sort_values(by='pvalue')

,pre_mean,post_mean,mean_diff,pvalue
country,,,,
CH,1536.321429,1023.892857,-512.428571,0.006941
MX,4975.464286,7033.214286,2057.750000,0.009670
IT,7651.571429,10193.428571,2541.857143,0.025730
FR,10385.250000,13635.000000,3249.750000,0.031843
ES,6648.642857,8660.571429,2011.928571,0.037522
UK,11213.535714,14196.428571,2982.892857,0.048490
DE,9856.750000,8013.964286,-1842.785714,0.081459
US,11921.035714,13077.464286,1156.428571,0.248874
CA,7468.428571,7880.428571,412.000000,0.351704


from above result, if we loose the significant level=0.1, then

daily spent in 'CH' and 'DE' are significantly decreased.
'MX','IT','FR','ES','UK', their daily spent are significant increased.
'US' and 'CA' has some improvement in daily spent, but NOT significant.

In [50]:
daily_stats_by_country.xs('n_purchase',level = 1).sort_values(by = 'pvalue')

,pre_mean,post_mean,mean_diff,pvalue
country,,,,
CH,26.821429,17.071429,-9.750000,0.003072
DE,232.142857,164.035714,-68.107143,0.011798
MX,126.464286,164.000000,37.535714,0.032430
IT,180.857143,227.928571,47.071429,0.057454
FR,244.142857,305.714286,61.571429,0.058996
ES,156.607143,194.500000,37.892857,0.072638
CA,177.142857,160.000000,-17.142857,0.233985
UK,264.285714,289.035714,24.750000,0.261183
US,282.285714,263.428571,-18.857143,0.307801


In [51]:
daily_stats_by_country.xs('n_customer',level = 1).sort_values(by = 'pvalue')

,pre_mean,post_mean,mean_diff,pvalue
country,,,,
CH,26.714286,17.071429,-9.642857,0.003142
DE,224.964286,163.250000,-61.714286,0.015665
MX,124.392857,163.107143,38.714286,0.026203
FR,236.500000,302.535714,66.035714,0.041124
IT,176.535714,226.107143,49.571429,0.043911
ES,153.392857,193.214286,39.821429,0.057954
UK,255.571429,286.321429,30.750000,0.204398
CA,173.285714,159.178571,-14.107143,0.268256
US,273.178571,261.107143,-12.071429,0.368860



from above result,

daily customers in 'CH' and 'DE' are significantly decreased.
'MX','IT','FR','ES', their daily customers are significant increased.

Answer question 1
Can you estimate the impact the program had on the site?

according to the analysis above, the program doesn't seem have significant impacts to the whole company as a whole.

however, based on each country, I find the program has different impact on different country:

the program fails in CH and DE, it significantly decrease the sales in these two countries.
the program succeeds in 'MX','IT','FR','ES', it significantly increase the sales.
the program doesn't seem have any significant effect on UK,CA,US, especially on CA and US.



Answer question 2
Based on the data, what would you suggest to do as a next step?

first I suggest perform more accurate A/B test (see question 3's answer) and collect more data, to study the impact of the program
since the program has different impact in different country, I suggest studying the reason of such difference. for example, does the program has any cultural conflicts in CH and DE?